In [1]:
import sys, os
sys.path.append(os.path.abspath(".."))

import xarray as xr
import matplotlib.pyplot as plt
import xesmf as xe
import ESMF

# === Project modules ===
from data_loading.load_data import load_dataset, load_ensemble_any_latlon
from data_loading.subset_time import get_common_time_range, subset_time
from data_loading.subset_region import subset_dataset, get_reference_extent
from utils.temporal_stats import aggregate_to_daily
from utils.bias_metrics import mean_error
# The new functions we'll test:
from data_loading.unify_grids import force_match_dimensions, regrid_to_ensemble

# === 1. Load Example Data ===
ensemble_pattern = os.path.join("..", "data", "total_precipitation_2017010*.nc")
ds_ensemble = load_ensemble_any_latlon(ensemble_pattern)

reference_file = os.path.join("..", "data", "SPARTACUS2-DAILY_RR_2017.nc")
ds_ref = xr.open_dataset(reference_file)

print("Original Ensemble:", ds_ensemble)
print("Original Reference:", ds_ref)

# === 2. Spatial Subset (Optional) ===
# If we want a smaller region, we find the ref extent and subset both
ref_bounds = get_reference_extent(ds_ref, lat_var="lat", lon_var="lon")
ds_ensemble_sub = subset_dataset(ds_ensemble, lat_var="lat", lon_var="lon", bounds=ref_bounds)
ds_ref_sub      = subset_dataset(ds_ref, lat_var="lat", lon_var="lon", bounds=ref_bounds)

# === 3. Temporal Subset (Optional) ===
common_time = get_common_time_range(ds_ensemble_sub, ds_ref_sub, time_var="time")
ds_ensemble_sub = subset_time(ds_ensemble_sub, time_bounds=common_time)
ds_ref_sub      = subset_time(ds_ref_sub, time_bounds=common_time)

# === 4. Daily Aggregation if Ensemble is Hourly ===
ds_ens_daily = aggregate_to_daily(ds_ensemble_sub, "precipitation", method="sum", compute_ens_mean=True)
ds_ref_daily = ds_ref_sub["RR"]  # Already daily

print("\nEnsemble daily shape:", ds_ens_daily.shape)
print("Reference daily shape:", ds_ref_daily.shape)

ModuleNotFoundError: No module named 'ESMF'